**Download Bert pretrained model + install dependencies**

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip multi_cased_L-12_H-768_A-12
!pip install bert-serving-server --no-deps
!pip install tensorflow==1.15

In [0]:
!pip install -U bert-serving-server[http]


In [0]:
import numpy as np
import pandas as pd
import json
from bert_serving.client import BertClient
from termcolor import colored
import io
topk = 10



**Import Knowledge Dataset**

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
dataset = pd.read_json('websites.json')
data = dataset[['title','contents','url']]
data

In [0]:
# import os
# os.system('cmd /k "bert-serving-start -num_worker=1 -model_dir=\content\multi_cased_L-12_H-768_A-12"')
# !pip install -U bert-serving-server[http]
# !pip install --upgrade ipykernel

# !nohup bert-serving-start -cpu -model_dir=.\multi_cased_L-12_H-768_A-12 >out.file2>&1&
# port_num='3399'
# get_ipython().system_raw(
#     'bert-serving-start -model_dir=\content\multi_cased_L-12_H-768_A-12 -http_port ' + port_num + ' &'
# )


In [0]:
# import json
# import requests
# def get_embeddings(texts):
#     headers = {
#         'content-type':'application/json'
#     }
#     data = {
#         "id":123,
#         "texts":texts,
#         "is_tokenized": False
#     }
#     data = json.dumps(data)
#     r = requests.post("http://localhost:" + port_num + "/encode", data=data, headers=headers).json()
#     return r['result']
# import socket
# socket.gethostbyname(socket.gethostname())

**Using BertClient to extract text embeddings**

In [0]:
!nohup bert-serving-start -model_dir=./multi_cased_L-12_H-768_A-12 > out.file 2>&1 &

from bert_serving.client import BertClient
bc = BertClient(check_length=False)


In [0]:
print (bc.encode(list(data['contents'])))

**Encoding our data**

In [0]:
doc_vecs = bc.encode(list(data['contents']))

**Smiliraities**

In [0]:
# while True:
query = input(colored('your Query: ', 'green'))
query_vec = bc.encode([query])[0]
# compute normalized dot product as score
score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
topk_idx = np.argsort(score)[::-1][:topk]
print('top %d articles similar to "%s"' % (topk, colored(query, 'green')))
for idx in topk_idx:
    print('> %s\t%s' % (colored('%.1f' % score[idx], 'cyan'), colored(data['title'][idx], 'yellow')))